# YOLO_v1

![](https://pic3.zhimg.com/80/v2-4a8db9d67ca78afe04d5610e31e6061a_1440w.jpg)

#### 目标检测要做什么事情。
目标检测就是要**找出图片中物体的bounding box（边界框），并判定框内物体的类别。**比如上图中有一只猫，那我们要将这只猫用一个bounding box框出来(bounding box可以用左上角坐标(x,y)和矩形的宽高(w，h)来表示)。


## yolo的思想（以论文举例）

![](https://pic2.zhimg.com/80/v2-fdfea5fcb4ff3ecc327758878e4ad6e1_1440w.jpg)
***
将输入图像按照模型的输出网格（比如7x7大小）进行划分，划分之后就有很多小cell了。我们再看图片中物体的中心是落在哪个cell里面，落在哪个cell哪个cell就负责预测这个物体。比如下图中，狗的中心落在了红色cell内，则这个cell负责预测狗。这么说可能不太容易理解，下面进行更具体的介绍。
***
实际上，“物体落在哪个cell，哪个cell就负责预测这个物体” 要分两个阶段来看，包括训练和测试。
* 训练阶段。在训练阶段，如果物体中心落在这个cell，那么就给这个cell打上这个物体的label（包括xywh和类别）。也就是说我们是通过这种方式来设置训练的label的。**换言之，我们在训练阶段，就教会cell要预测图像中的哪个物体。**
* 测试阶段。因为你在训练阶段已经教会了cell去预测中心落在该cell中的物体，那么cell自然也会这么做。


## yolo的模型

### 模型架构
![](https://pic4.zhimg.com/80/v2-5d099287b1237fa975b1c19bacdfc07f_1440w.jpg)

从图中可以看到，yolo网络的输出的网格是7x7大小的，另外，输出的channel数目为30。一个cell内，前20个元素是类别概率值，然后2个元素是边界框confidence，最后8个元素是边界框的 (x, y,w,h)。

![](https://pic3.zhimg.com/80/v2-ccdd48a4323783358a0cd31dbf86b52a_1440w.jpg)

也就是说，每个cell有两个predictor，每个predictor(bounding box)分别预测一个bounding box的xywh和相应的confidence。但分类部分的预测却是共享的。正因为这个，同个cell是没办法预测多个目标的。

现在考虑两个问题:
1. 假设类别预测不是共享的，cell中两个predictor都有各自的类别预测，这样能否在一个cell中预测两个目标？
2. 为什么要预测两个bounding box？

对于第一个问题，答案是否定的。如果一个cell要预测两个目标，那么这两个predictor要怎么分工预测这两个目标？谁负责谁？不知道，所以没办法预测。而像faster rcnn这类算法，可以根据anchor与ground truth的IOU大小来安排anchor负责预测哪个物体，所以后来yolo2也采用了anchor思想，同个cell才能预测多个目标。

对于第二个问题，既然我们一个cell只能预测一个目标，为什么还要预测两个bounding box（或者更多）？这个还是要从训练阶段怎么给两个predictor安排训练目标来说。  
**在训练的时候会在线地计算每个predictor预测的bounding box和ground truth的IOU，计算出来的IOU大的那个predictor，就会负责预测这个物体，另外一个则不预测。** 
  这么做有什么好处？我的理解是，这样做的话，实际上有两个predictor来一起进行预测，然后网络会在线选择预测得好的那个predictor（也就是IOU大）来进行预测。通俗一点说，就是我找一堆人来并行地干一件事，然后我选干的最好的那个。
  
### 模型输出的意义

#### Confidence预测
首先看cell预测的bounding box中condifence这个维度。confidence表示：cell预测的bounding box包含一个物体的置信度有多高并且该bounding box预测准确度有多大，用公式表示为：![](https://www.zhihu.com/equation?tex=Pr%28Object%29+%2A+IOU%5E%7Btruth%7D_%7Bpred%7D)

这个也要分两个阶段来考虑。
![](1.png)

**我个人的理解，卷积神经网络会训练出，针对每一个有object可能的grid，应该要有多大的bounding box

![](2.png)
![](3.png)

### 训练阶段
![](4.png)